In [1]:
%matplotlib inline
%load_ext line_profiler
import numpy as np
import scipy as sp
import scipy.sparse as spar
import scipy.special as spec
import sys
from matplotlib import pyplot as plt
from lda import LDA, _doc_update, _slice_doc_update
from sklearn.decomposition import LatentDirichletAllocation as SKLDA

import pickle
import cProfile

In [2]:
from data.datafile import AADataFile
dfile = pickle.load(open("data/datafile.pkl"))

In [3]:
dt = dfile.DT
te = dfile.TE

In [4]:
f = te.toarray().argmax(axis=1)

In [10]:
lda = LDA(K=10, n_jobs=8, nr_em_epochs=20)
%time b_uni, g_uni = lda.fit(dt)

Epoch: 0
Perplexity: 1.63261084698
Epoch: 1
Perplexity: 325.745739171
Epoch: 2
Perplexity: 298.398471194
Epoch: 3
Perplexity: 277.064323152
Epoch: 4
Perplexity: 261.40464915
Epoch: 5
Perplexity: 250.89628224
Epoch: 6
Perplexity: 244.016180078
Epoch: 7
Perplexity: 239.499604046
Epoch: 8
Perplexity: 236.577933169
Epoch: 9
Perplexity: 234.543853573
Epoch: 10
Perplexity: 233.020824677
Epoch: 11
Perplexity: 231.820478723
Epoch: 12
Perplexity: 230.901512006
Epoch: 13
Perplexity: 230.21296433
Epoch: 14
Perplexity: 229.66801685
Epoch: 15
Perplexity: 229.221419607
Epoch: 16
Perplexity: 228.858723131
Epoch: 17
Perplexity: 228.534450907
Epoch: 18
Perplexity: 228.258660649
Epoch: 19
Perplexity: 228.001226572
CPU times: user 1.87 s, sys: 1.67 s, total: 3.54 s
Wall time: 1min 59s


In [7]:
vocab = dfile.vocabulary

In [8]:
lookup = lambda x : [k[0] for k in vocab if k[1] == x]

def topic_summaries(b):
    bs = b.argsort()[:,-50:]
    for i in range(10):
        print "Topic", i
        words = []
        for j in range(bs.shape[1]):
            words.append(lookup(bs[i,j])[0])
        print " ".join(words)

In [11]:
topic_summaries(b_uni)

Topic 0
yaptığı trafik şehit verildi vatandaşlar sarıkamış yer yeni devam nun bağlı na edildi ekipleri deniz yılbaşı türkiye bazı günü il yaklaşık olumsuz saat sonucu etti eğitime in kişi tarafından ara sağlık ilçesinde nde nda eğitim nin olan gelen gün yapılan açıklamada nın ın yoğun yağışı istanbul ardından hava muhabir kar
Topic 1
ukrayna ab 15 sosyal ayında su yılda son dönemde gelecek aynı 2014 olacak genel büyük oldu nin toplam çin yüksek 30 dolar asgari yaklaşık enerji ilişkin 10 ülke ticaret te geçen yılın olan nın yılı yılında ekonomik yeni ocak lira olarak milyar türkiye rusya 2015 yıl 2016 milyon yüzde bin
Topic 2
bilgiye imha yönelik olay getirildi gelen zarar güçleri arama edildi olduğu ekipleri iş alınan çıkma sınır sokağa eden in operasyonlarda operasyonda iki yapılan ele bölücü ardından teröristlerin açıklamada mahallesi pkk silopi devam şırnak jandarma patlayıcı yapımı terörist diyarbakır cizre şehit etkisiz sur el ilçesinde ın hale polis örgütü güvenlik terör
Topic 3


# Multimodal Starts Here!

In [5]:
lda = LDA(K=10, n_jobs=8, nr_em_epochs=20)
%time b, g, e = lda.fit(dt, f)

Epoch: 0
Perplexity: 1.67907689262
Epoch: 1
Perplexity: 359.131142056
Epoch: 2
Perplexity: 359.307590519
Epoch: 3
Perplexity: 343.016780133
Epoch: 4
Perplexity: 325.265367587
Epoch: 5
Perplexity: 315.536865259
Epoch: 6
Perplexity: 309.286199933
Epoch: 7
Perplexity: 305.615378896
Epoch: 8
Perplexity: 303.072672946
Epoch: 9
Perplexity: 301.389319777
Epoch: 10
Perplexity: 300.435784124
Epoch: 11
Perplexity: 299.882553233
Epoch: 12
Perplexity: 299.41994288
Epoch: 13
Perplexity: 298.891643543
Epoch: 14
Perplexity: 298.362570784
Epoch: 15
Perplexity: 297.874961706
Epoch: 16
Perplexity: 297.542599662
Epoch: 17
Perplexity: 297.294460161
Epoch: 18
Perplexity: 297.037783128
Epoch: 19
Perplexity: 296.795661696
CPU times: user 1.9 s, sys: 1.64 s, total: 3.53 s
Wall time: 1min 57s


In [9]:
topic_summaries(b)

Topic 0
kaydedildi belediye memuru harekat kararı tespit mahsur isabet adliyeye infilak askeri müdahale merkezi silahlı merkez hasar itfaiye merkezli adli müdürlüğüne cenaze devlet zarar vatandaşlar tedavi şehidin asker mensubu cenazesi imha vatandaşların caddesi sokağa mücadele şüpheli tarafından hayatını cumhuriyet şube müdürü devam sevk muhabir mahallesi sur müdürlüğü diyarbakır hale şehit emniyet
Topic 1
hdp 2014 25 mustafa 14 ele 20 ahmet 12 13 17 100 yi 30 pkk rus 11 mehmet 15 iran irak na ankara arabistan den daki tüm nun suudi dan 10 nda abd daeş davutoğlu israil erdoğan nde türk istanbul te suriye 2016 rusya 2015 aa nın ın nin türkiye
Topic 2
gerektiğini vurgulayan ardından yapan etmek getiren bin böyle bulundu bazı ederek ilişkin anlatan iş iki belirtti ortaya konusunda bakanı bütün olduğu şunları değil yıl bizim in belirten son önemli genel belirterek başbakan olan kullandı ye ilgili dile açıklamada etti büyük var yeni başkanı eden yaptığı konuştu olarak söyledi olduğunu ded

In [12]:
from sklearn.metrics import adjusted_rand_score as ARI, adjusted_mutual_info_score as AMI

In [18]:
unimodal = np.argmax(g_uni, 0)
multimodal = np.argmax(g, 0)

In [21]:
cat = dfile.categories

In [24]:
print "Unimodal Rand: ", ARI(unimodal, cat)
print "Unimodal Adj MI: ", AMI(unimodal, cat)

print "Multimodal Rand: ", ARI(multimodal, cat)
print "Multimodal Adj MI: ", AMI(multimodal, cat)

Unimodal Rand:  0.216581773357
Unimodal Adj MI:  0.291553303472
Multimodal Rand:  0.0618414672066
Multimodal Adj MI:  0.135851843571
